In [1]:
# CA
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

In [2]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [3]:
pd.options.display.max_columns = 50

h = 28 
max_lags = 57
tr_last = 1913
fday = datetime(2016,4, 25) 
fday

datetime.datetime(2016, 4, 25, 0, 0)

In [4]:
prices = pd.read_csv("sell_prices.csv", dtype = PRICE_DTYPES)
prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [5]:
def create_dt(is_train = True, nrows = None, first_day = 1200, validate = True, store_id = None):
    prices = pd.read_csv("sell_prices.csv", dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()
    
    if not store_id is None:
        prices = prices.loc[prices['store_id'] == store_id]
        #prices = prices.drop(columns=['store_id'])
            
    cal = pd.read_csv("calendar.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()
    
    start_day = max(1 if is_train  else tr_last-max_lags, first_day)
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    if validate:
        dt = pd.read_csv("sales_train_validation.csv", 
                         nrows = nrows, usecols = catcols + numcols, dtype = dtype)
    else:
        dt = pd.read_csv("sales_train_evaluation.csv", 
                         nrows = nrows, usecols = catcols + numcols, dtype = dtype)
    
    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()
    
    if not is_train:
        for day in range(tr_last+1, tr_last+ 28 +1):
            dt[f"d_{day}"] = np.nan
    
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    
    dt = dt.merge(cal, on= "d", copy = False)
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    
    return dt

In [6]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    
    
    date_features = {
        
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [7]:
%%time
FIRST_DAY = 1
STORE_ID = 0
df = create_dt(is_train=True, first_day= FIRST_DAY, store_id = STORE_ID)
df.shape

CPU times: user 28.8 s, sys: 9.84 s, total: 38.7 s
Wall time: 39.9 s


(4702895, 22)

In [8]:
df.shape

(4702895, 22)

In [9]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_1,12.0,2011-01-29,11101,2,1,1,2011,0,0,0,0,0.0,0.0,0.0,0.46
1,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_2,15.0,2011-01-30,11101,3,2,1,2011,0,0,0,0,0.0,0.0,0.0,0.46
2,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_3,0.0,2011-01-31,11101,1,3,1,2011,0,0,0,0,0.0,0.0,0.0,0.46
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_4,0.0,2011-02-01,11101,5,4,2,2011,0,0,0,0,1.0,1.0,0.0,0.46
4,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_5,0.0,2011-02-02,11101,6,5,2,2011,0,0,0,0,1.0,0.0,1.0,0.46


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4702895 entries, 0 to 4702894
Data columns (total 22 columns):
id              object
item_id         int16
dept_id         int16
store_id        int16
cat_id          int16
state_id        int16
d               object
sales           float32
date            datetime64[ns]
wm_yr_wk        int16
weekday         int16
wday            int16
month           int16
year            int16
event_name_1    int16
event_type_1    int16
event_name_2    int16
event_type_2    int16
snap_CA         float32
snap_TX         float32
snap_WI         float32
sell_price      float32
dtypes: datetime64[ns](1), float32(5), int16(14), object(2)
memory usage: 358.8+ MB


In [11]:
%%time

create_fea(df)
df.shape

CPU times: user 14.3 s, sys: 971 ms, total: 15.2 s
Wall time: 12.3 s


(4702895, 31)

In [12]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
0,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_1,12.0,2011-01-29,11101,2,1,1,2011,0,0,0,0,0.0,0.0,0.0,0.46,NaN,NaN,NaN,NaN,NaN,NaN,4,1,29
1,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_2,15.0,2011-01-30,11101,3,2,1,2011,0,0,0,0,0.0,0.0,0.0,0.46,NaN,NaN,NaN,NaN,NaN,NaN,4,1,30
2,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_3,0.0,2011-01-31,11101,1,3,1,2011,0,0,0,0,0.0,0.0,0.0,0.46,NaN,NaN,NaN,NaN,NaN,NaN,5,1,31
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_4,0.0,2011-02-01,11101,5,4,2,2011,0,0,0,0,1.0,1.0,0.0,0.46,NaN,NaN,NaN,NaN,NaN,NaN,5,1,1
4,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_5,0.0,2011-02-02,11101,6,5,2,2011,0,0,0,0,1.0,0.0,1.0,0.46,NaN,NaN,NaN,NaN,NaN,NaN,5,1,2


In [13]:
df.dropna(inplace = True)
print(df.shape)
df.head()

(4535200, 31)


,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
67234,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_56,0.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,0.0,0.0,0.42,0.0,0.0,2.000000,4.000000,1.642857,4.535714,12,1,25
67241,HOBBIES_1_009_CA_1_validation,8,0,0,0,0,d_56,0.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,0.0,0.0,1.77,0.0,0.0,2.000000,2.000000,2.142857,2.178571,12,1,25
67248,HOBBIES_1_010_CA_1_validation,9,0,0,0,0,d_56,0.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,0.0,0.0,3.17,0.0,0.0,0.000000,0.142857,0.035714,0.214286,12,1,25
67255,HOBBIES_1_012_CA_1_validation,11,0,0,0,0,d_56,1.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,0.0,0.0,6.27,1.0,1.0,0.428571,0.857143,0.535714,0.678571,12,1,25
67262,HOBBIES_1_015_CA_1_validation,14,0,0,0,0,d_56,1.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,0.0,0.0,0.72,2.0,1.0,5.428571,8.000000,6.142857,4.321429,12,1,25


In [14]:
df["store_id"].value_counts()

0    4535200
Name: store_id, dtype: int64

In [15]:
# change here according to store_id



df_CA1 = df.drop(columns = ['state_id', 'snap_TX', 'snap_WI'])

In [16]:
print(df_CA1.shape)
df_CA1.head()

(4535200, 28)


,id,item_id,dept_id,store_id,cat_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
67234,HOBBIES_1_008_CA_1_validation,7,0,0,0,d_56,0.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,0.42,0.0,0.0,2.000000,4.000000,1.642857,4.535714,12,1,25
67241,HOBBIES_1_009_CA_1_validation,8,0,0,0,d_56,0.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,1.77,0.0,0.0,2.000000,2.000000,2.142857,2.178571,12,1,25
67248,HOBBIES_1_010_CA_1_validation,9,0,0,0,d_56,0.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,3.17,0.0,0.0,0.000000,0.142857,0.035714,0.214286,12,1,25
67255,HOBBIES_1_012_CA_1_validation,11,0,0,0,d_56,1.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,6.27,1.0,1.0,0.428571,0.857143,0.535714,0.678571,12,1,25
67262,HOBBIES_1_015_CA_1_validation,14,0,0,0,d_56,1.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,0.72,2.0,1.0,5.428571,8.000000,6.142857,4.321429,12,1,25


In [17]:
# df_CA_2
STORE_ID = 1
df = create_dt(is_train=True, first_day= FIRST_DAY, store_id = STORE_ID)
create_fea(df)
df.dropna(inplace = True)
df_CA2 = df.drop(columns = ['state_id', 'snap_TX', 'snap_WI'])
df_CA2.head()

,id,item_id,dept_id,store_id,cat_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
57182,HOBBIES_1_004_CA_2_validation,3,0,1,0,d_56,1.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,4.34,7.0,0.0,4.571429,3.428571,4.035714,3.928571,12,1,25
57189,HOBBIES_1_009_CA_2_validation,8,0,1,0,d_56,1.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,1.77,0.0,4.0,1.285714,2.000000,1.642857,1.321429,12,1,25
57196,HOBBIES_1_010_CA_2_validation,9,0,1,0,d_56,0.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,2.97,2.0,1.0,1.000000,1.000000,0.892857,0.750000,12,1,25
57203,HOBBIES_1_012_CA_2_validation,11,0,1,0,d_56,1.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,6.27,0.0,0.0,0.285714,0.714286,0.750000,1.107143,12,1,25
57210,HOBBIES_1_015_CA_2_validation,14,0,1,0,d_56,0.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,0.72,3.0,0.0,3.714286,2.285714,1.892857,1.214286,12,1,25


In [18]:
# df_CA_3
STORE_ID = 2
df = create_dt(is_train=True, first_day= FIRST_DAY, store_id = STORE_ID)
create_fea(df)
df.dropna(inplace = True)
df_CA3 = df.drop(columns = ['state_id', 'snap_TX', 'snap_WI'])
df_CA3.head()

,id,item_id,dept_id,store_id,cat_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
66758,HOBBIES_1_008_CA_3_validation,7,0,2,0,d_56,44.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,0.42,30.0,8.0,29.714285,23.285715,18.000000,18.392857,12,1,25
66765,HOBBIES_1_009_CA_3_validation,8,0,2,0,d_56,0.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,1.56,0.0,0.0,1.142857,1.428571,1.500000,1.500000,12,1,25
66772,HOBBIES_1_010_CA_3_validation,9,0,2,0,d_56,1.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,2.97,0.0,0.0,0.714286,0.714286,0.535714,0.678571,12,1,25
66779,HOBBIES_1_012_CA_3_validation,11,0,2,0,d_56,1.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,6.27,1.0,3.0,1.000000,1.857143,1.142857,1.571429,12,1,25
66786,HOBBIES_1_015_CA_3_validation,14,0,2,0,d_56,6.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,0.72,2.0,14.0,6.714286,8.428572,8.750000,7.357143,12,1,25


In [19]:
# df_CA_4
STORE_ID = 3
df = create_dt(is_train=True, first_day= FIRST_DAY, store_id = STORE_ID)
create_fea(df)
df.dropna(inplace = True)
df_CA4 = df.drop(columns = ['state_id','snap_TX', 'snap_WI'])
df_CA4.head()

,id,item_id,dept_id,store_id,cat_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
59506,HOBBIES_1_004_CA_4_validation,3,0,3,0,d_56,1.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,4.34,1.0,2.0,1.000000,1.142857,0.821429,0.892857,12,1,25
59513,HOBBIES_1_010_CA_4_validation,9,0,3,0,d_56,1.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,2.97,0.0,0.0,0.142857,0.428571,0.250000,0.357143,12,1,25
59527,HOBBIES_1_015_CA_4_validation,14,0,3,0,d_56,15.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,0.72,9.0,6.0,5.000000,3.714286,6.821429,5.750000,12,1,25
59534,HOBBIES_1_020_CA_4_validation,19,0,3,0,d_56,0.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,10.98,0.0,0.0,0.142857,0.142857,0.142857,0.321429,12,1,25
59541,HOBBIES_1_022_CA_4_validation,21,0,3,0,d_56,0.0,2011-03-25,11108,0,7,3,2011,0,0,0,0,0.0,6.98,0.0,0.0,0.000000,0.000000,0.000000,0.107143,12,1,25


In [20]:
df_CA = pd.concat([df_CA1, df_CA2, df_CA3, df_CA4])
df_CA1 = df_CA

In [21]:
cat_feats = ['item_id', 'dept_id', 'cat_id', 'store_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday", "year"]
train_cols = df_CA1.columns[~df_CA1.columns.isin(useless_cols)]
X_train = df_CA1[train_cols]
y_train = df_CA1["sales"]

In [22]:
X_train.head()

,item_id,dept_id,store_id,cat_id,wday,month,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
67234,7,0,0,0,7,3,0,0,0,0,0.0,0.42,0.0,0.0,2.000000,4.000000,1.642857,4.535714,12,1,25
67241,8,0,0,0,7,3,0,0,0,0,0.0,1.77,0.0,0.0,2.000000,2.000000,2.142857,2.178571,12,1,25
67248,9,0,0,0,7,3,0,0,0,0,0.0,3.17,0.0,0.0,0.000000,0.142857,0.035714,0.214286,12,1,25
67255,11,0,0,0,7,3,0,0,0,0,0.0,6.27,1.0,1.0,0.428571,0.857143,0.535714,0.678571,12,1,25
67262,14,0,0,0,7,3,0,0,0,0,0.0,0.72,2.0,1.0,5.428571,8.000000,6.142857,4.321429,12,1,25


In [23]:
X_train.shape

(17547018, 21)

In [24]:
%%time

np.random.seed(777)

fake_valid_inds = np.random.choice(X_train.index.values, 500_000, replace = False)
train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)
train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds], 
                         categorical_feature=cat_feats, free_raw_data=False)
fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
                              categorical_feature=cat_feats,
                 free_raw_data=False)
# This is a random sample, we're not gonna apply any time series train-test-split tricks here!

CPU times: user 3min 24s, sys: 2min 47s, total: 6min 12s
Wall time: 3min 21s


In [25]:
fake_valid_inds.shape

(500000,)

In [26]:
train_inds.shape

(4157197,)

In [27]:
del df, X_train, y_train, fake_valid_inds,train_inds ; gc.collect()

60

In [28]:
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 1000,
    'num_leaves': 128,
    "min_data_in_leaf": 100,
}

In [29]:
%%time

m_lgb = lgb.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=20) 

/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[20]	valid_0's rmse: 3.18884
[40]	valid_0's rmse: 2.82497
[60]	valid_0's rmse: 2.72657
[80]	valid_0's rmse: 2.69705
[100]	valid_0's rmse: 2.68498
[120]	valid_0's rmse: 2.6725
[140]	valid_0's rmse: 2.66235
[160]	valid_0's rmse: 2.65617
[180]	valid_0's rmse: 2.64617
[200]	valid_0's rmse: 2.64204
[220]	valid_0's rmse: 2.63126
[240]	valid_0's rmse: 2.62502
[260]	valid_0's rmse: 2.61874
[280]	valid_0's rmse: 2.61517
[300]	valid_0's rmse: 2.6095
[320]	valid_0's rmse: 2.60381
[340]	valid_0's rmse: 2.5982
[360]	valid_0's rmse: 2.59199
[380]	valid_0's rmse: 2.58922
[400]	valid_0's rmse: 2.58628
[420]	valid_0's rmse: 2.58272
[440]	valid_0's rmse: 2.58073
[460]	valid_0's rmse: 2.57677
[480]	valid_0's rmse: 2.57338
[500]	valid_0's rmse: 2.57033
[520]	valid_0's rmse: 2.56437
[540]	valid_0's rmse: 2.56189
[560]	valid_0's rmse: 2.5604
[580]	valid_0's rmse: 2.55864
[600]	valid_0's rmse: 2.55558
[620]	valid_0's rmse: 2.55384
[640]	valid_0's rmse: 2.55057
[660]	valid_0's rmse: 2.54903
[680]	valid_0's rm

In [31]:
m_lgb.save_model("model_CA.lgb")

In [30]:
#m_lgb = lgb.Booster(model_file= "model_" + str(STORE_ID) + ".lgb")

In [31]:
m_lgb

In [56]:
%%time

alphas = [1.028, 1.023, 1.018]
#alphas = [1.025, 1.017, 1.013]
weights = [1/len(alphas)]*len(alphas)
sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):

    te = create_dt(False, store_id = 0)
    mask = te["store_id"] == 0
    te_1 = te[mask]
    
    te = create_dt(False, store_id = 1)
    mask = te["store_id"] == 1
    te_2 = te[mask]
    
    te = create_dt(False, store_id = 2)
    mask = te["store_id"] == 2
    te_3 = te[mask]
    
    te = create_dt(False, store_id = 3)
    mask = te["store_id"] == 3
    te_4 = te[mask]
    
    te_CA1 = pd.concat([te_1, te_2, te_3, te_4])
    te_CA1 = te_CA1.drop("snap_TX", axis = 1)
    te_CA1 = te_CA1.drop("snap_WI", axis = 1)
    
    cols = [f"F{i}" for i in range(1,29)]

    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)
        print(tdelta, day)
        tst = te_CA1[(te_CA1.date >= day - timedelta(days=max_lags)) & (te_CA1.date <= day)].copy()
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols]
        te_CA1.loc[te_CA1.date == day, "sales"] = alpha*m_lgb.predict(tst) # magic multiplier by 
        

    te_sub = te_CA1.loc[te_CA1.date >= fday, ["id", "sales"]].copy()

    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
    te_sub.fillna(0., inplace = True)
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    #te_sub.to_csv(f"submission_{icount}.csv",index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)


sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("CA.csv",index=False)

0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2016-05-10 00:00:00
16 2016-05-11 00:00:00
17 2016-05-12 00:00:00
18 2016-05-13 00:00:00
19 2016-05-14 00:00:00
20 2016-05-15 00:00:00
21 2016-05-16 00:00:00
22 2016-05-17 00:00:00
23 2016-05-18 00:00:00
24 2016-05-19 00:00:00
25 2016-05-20 00:00:00
26 2016-05-21 00:00:00
27 2016-05-22 00:00:00
0 1.028 0.3333333333333333
0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2

In [57]:
sub.shape

(24392, 29)

In [58]:
evaluation_set = pd.read_csv("sales_train_evaluation.csv")
evaluation_set.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,...,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,0,3,5,0,0,1,1,0,2,1,2,2,1,0,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,2,1,0,0,0,0,2,1,3,0,0,1,0,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,4,1,6,4,0,0,0,2,2,4,2,1,1,1,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,1,0,3,2,3,1,1,3,2,3,2,2,2,2,0,0,0,2,1,0,0,2,1,0


In [59]:
mask = evaluation_set["state_id"] == "CA"
CA_1_eva = evaluation_set[mask]
CA_1_eva.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,...,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,0,3,5,0,0,1,1,0,2,1,2,2,1,0,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,2,1,0,0,0,0,2,1,3,0,0,1,0,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,4,1,6,4,0,0,0,2,2,4,2,1,1,1,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,1,0,3,2,3,1,1,3,2,3,2,2,2,2,0,0,0,2,1,0,0,2,1,0


In [60]:
CA_1_eva = CA_1_eva.sort_values('id')
CA_1_eva

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,...,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
1612,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,0,0,1,4,2,0,2,0,0,0,0,3,1,3,0,2,1,2,...,0,0,1,1,0,6,1,0,3,0,0,0,2,0,0,2,3,1,0,0,0,1,0,0,0
4661,FOODS_1_001_CA_2_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_2,CA,2,0,0,0,1,0,8,0,4,1,2,1,1,3,4,1,2,2,2,...,0,0,1,4,0,0,0,0,0,0,0,1,2,1,0,1,0,0,1,1,0,0,1,2,0
7710,FOODS_1_001_CA_3_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_3,CA,1,2,1,1,1,2,0,1,1,1,0,0,3,3,2,7,1,2,4,...,0,8,1,0,0,1,0,0,1,1,0,0,0,0,0,1,2,2,0,0,1,0,3,2,2
10759,FOODS_1_001_CA_4_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_4,CA,0,1,1,1,1,1,0,0,0,0,0,0,1,2,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0
1613,FOODS_1_002_CA_1_evaluation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,0,1,0,1,1,1,0,0,0,0,1,1,0,2,0,1,0,0,0,...,0,0,1,0,1,3,0,0,1,0,2,1,0,0,1,2,1,0,1,1,1,0,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10757,HOUSEHOLD_2_515_CA_4_evaluation,HOUSEHOLD_2_515,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1611,HOUSEHOLD_2_516_CA_1_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,CA_1,CA,0,1,0,0,1,0,0,1,0,2,0,1,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
4660,HOUSEHOLD_2_516_CA_2_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,CA_2,CA,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,3,0
7709,HOUSEHOLD_2_516_CA_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,CA_3,CA,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0


In [61]:
CA_1_eva = CA_1_eva.iloc[:, -28:]
CA_1_eva

,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
1612,2,0,0,0,0,1,1,0,6,1,0,3,0,0,0,2,0,0,2,3,1,0,0,0,1,0,0,0
4661,0,3,0,0,0,1,4,0,0,0,0,0,0,0,1,2,1,0,1,0,0,1,1,0,0,1,2,0
7710,1,0,1,0,8,1,0,0,1,0,0,1,1,0,0,0,0,0,1,2,2,0,0,1,0,3,2,2
10759,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0
1613,0,0,0,0,0,1,0,1,3,0,0,1,0,2,1,0,0,1,2,1,0,1,1,1,0,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10757,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1611,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
4660,0,2,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,3,0
7709,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0


In [62]:
sub.shape

(24392, 29)

In [63]:
3049*4

12196

In [64]:
pred_period1 = sub[0:12196]
pred_period1

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.978327,0.886317,0.885267,0.840258,1.144252,1.302439,1.371132,1.062059,0.993374,1.026958,1.008744,1.177302,1.428466,1.393643,1.052061,0.958091,0.966247,0.941703,1.095820,1.405988,1.280506,1.030439,0.922642,0.906377,0.902626,1.101821,1.349500,1.300018
1,FOODS_1_001_CA_2_validation,0.956117,1.017991,0.915236,1.284247,1.277013,1.508155,1.763588,1.090965,1.083976,1.063215,1.071434,1.326841,1.702894,1.514765,1.219710,1.163053,1.165274,1.146498,1.462949,1.877415,1.382356,1.192399,1.119652,1.118806,1.149937,1.352882,1.736652,1.458059
2,FOODS_1_001_CA_3_validation,1.141158,1.060389,1.009209,0.978844,1.064277,1.173482,1.259915,1.026248,1.060106,1.052838,0.999624,1.290200,1.573873,1.583603,1.184410,1.090997,1.060624,1.043204,1.194069,1.567379,1.400124,1.187711,1.124349,1.113409,1.090218,1.206241,1.428213,1.466147
3,FOODS_1_001_CA_4_validation,0.408730,0.359662,0.331148,0.332599,0.407499,0.433837,0.602332,0.402484,0.428838,0.420318,0.374566,0.360211,0.422587,0.362438,0.368652,0.373735,0.383930,0.394509,0.395899,0.455351,0.481506,0.392743,0.369007,0.370575,0.372364,0.406188,0.461010,0.474532
4,FOODS_1_002_CA_1_validation,0.422404,0.430875,0.449946,0.520700,0.453983,0.622715,0.608194,0.479387,0.496449,0.435287,0.424406,0.440619,0.651719,0.527341,0.486947,0.455321,0.453260,0.450625,0.463734,0.625805,0.650339,0.482945,0.457457,0.470726,0.482871,0.496654,0.604415,0.604536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12191,HOUSEHOLD_2_515_CA_4_validation,0.128544,0.133970,0.139405,0.140420,0.131640,0.194804,0.157752,0.140396,0.156087,0.155214,0.138554,0.158417,0.162208,0.109328,0.124815,0.128203,0.134872,0.140703,0.132760,0.147825,0.130593,0.117594,0.121420,0.127832,0.134449,0.133994,0.165792,0.152036
12192,HOUSEHOLD_2_516_CA_1_validation,0.206858,0.195634,0.188470,0.211799,0.307360,0.396261,0.305189,0.250490,0.225812,0.219734,0.255343,0.263959,0.351914,0.344115,0.231780,0.219014,0.218169,0.216522,0.284538,0.336489,0.341310,0.233627,0.217282,0.211336,0.218080,0.302132,0.351668,0.340154
12193,HOUSEHOLD_2_516_CA_2_validation,0.199442,0.200549,0.202226,0.199433,0.242613,0.334718,0.305292,0.212616,0.207093,0.199566,0.237868,0.266541,0.302887,0.346094,0.214026,0.207233,0.204759,0.211739,0.289037,0.343196,0.324581,0.207095,0.198362,0.199624,0.203499,0.278499,0.285667,0.282954
12194,HOUSEHOLD_2_516_CA_3_validation,0.116540,0.113957,0.118031,0.110473,0.129658,0.165444,0.144743,0.146232,0.143333,0.139504,0.167418,0.173569,0.175563,0.164022,0.145093,0.143160,0.142121,0.140556,0.161535,0.170858,0.160630,0.136316,0.132347,0.135679,0.139172,0.164795,0.173509,0.163122


In [65]:
pred_period1 = pred_period1.sort_values('id')
pred_period1 

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.978327,0.886317,0.885267,0.840258,1.144252,1.302439,1.371132,1.062059,0.993374,1.026958,1.008744,1.177302,1.428466,1.393643,1.052061,0.958091,0.966247,0.941703,1.095820,1.405988,1.280506,1.030439,0.922642,0.906377,0.902626,1.101821,1.349500,1.300018
1,FOODS_1_001_CA_2_validation,0.956117,1.017991,0.915236,1.284247,1.277013,1.508155,1.763588,1.090965,1.083976,1.063215,1.071434,1.326841,1.702894,1.514765,1.219710,1.163053,1.165274,1.146498,1.462949,1.877415,1.382356,1.192399,1.119652,1.118806,1.149937,1.352882,1.736652,1.458059
2,FOODS_1_001_CA_3_validation,1.141158,1.060389,1.009209,0.978844,1.064277,1.173482,1.259915,1.026248,1.060106,1.052838,0.999624,1.290200,1.573873,1.583603,1.184410,1.090997,1.060624,1.043204,1.194069,1.567379,1.400124,1.187711,1.124349,1.113409,1.090218,1.206241,1.428213,1.466147
3,FOODS_1_001_CA_4_validation,0.408730,0.359662,0.331148,0.332599,0.407499,0.433837,0.602332,0.402484,0.428838,0.420318,0.374566,0.360211,0.422587,0.362438,0.368652,0.373735,0.383930,0.394509,0.395899,0.455351,0.481506,0.392743,0.369007,0.370575,0.372364,0.406188,0.461010,0.474532
4,FOODS_1_002_CA_1_validation,0.422404,0.430875,0.449946,0.520700,0.453983,0.622715,0.608194,0.479387,0.496449,0.435287,0.424406,0.440619,0.651719,0.527341,0.486947,0.455321,0.453260,0.450625,0.463734,0.625805,0.650339,0.482945,0.457457,0.470726,0.482871,0.496654,0.604415,0.604536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12191,HOUSEHOLD_2_515_CA_4_validation,0.128544,0.133970,0.139405,0.140420,0.131640,0.194804,0.157752,0.140396,0.156087,0.155214,0.138554,0.158417,0.162208,0.109328,0.124815,0.128203,0.134872,0.140703,0.132760,0.147825,0.130593,0.117594,0.121420,0.127832,0.134449,0.133994,0.165792,0.152036
12192,HOUSEHOLD_2_516_CA_1_validation,0.206858,0.195634,0.188470,0.211799,0.307360,0.396261,0.305189,0.250490,0.225812,0.219734,0.255343,0.263959,0.351914,0.344115,0.231780,0.219014,0.218169,0.216522,0.284538,0.336489,0.341310,0.233627,0.217282,0.211336,0.218080,0.302132,0.351668,0.340154
12193,HOUSEHOLD_2_516_CA_2_validation,0.199442,0.200549,0.202226,0.199433,0.242613,0.334718,0.305292,0.212616,0.207093,0.199566,0.237868,0.266541,0.302887,0.346094,0.214026,0.207233,0.204759,0.211739,0.289037,0.343196,0.324581,0.207095,0.198362,0.199624,0.203499,0.278499,0.285667,0.282954
12194,HOUSEHOLD_2_516_CA_3_validation,0.116540,0.113957,0.118031,0.110473,0.129658,0.165444,0.144743,0.146232,0.143333,0.139504,0.167418,0.173569,0.175563,0.164022,0.145093,0.143160,0.142121,0.140556,0.161535,0.170858,0.160630,0.136316,0.132347,0.135679,0.139172,0.164795,0.173509,0.163122


In [66]:
pred_period1 = pred_period1.drop(columns='id')
pred_period1.columns = CA_1_eva.columns

In [67]:
pred_period1

,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,0.978327,0.886317,0.885267,0.840258,1.144252,1.302439,1.371132,1.062059,0.993374,1.026958,1.008744,1.177302,1.428466,1.393643,1.052061,0.958091,0.966247,0.941703,1.095820,1.405988,1.280506,1.030439,0.922642,0.906377,0.902626,1.101821,1.349500,1.300018
1,0.956117,1.017991,0.915236,1.284247,1.277013,1.508155,1.763588,1.090965,1.083976,1.063215,1.071434,1.326841,1.702894,1.514765,1.219710,1.163053,1.165274,1.146498,1.462949,1.877415,1.382356,1.192399,1.119652,1.118806,1.149937,1.352882,1.736652,1.458059
2,1.141158,1.060389,1.009209,0.978844,1.064277,1.173482,1.259915,1.026248,1.060106,1.052838,0.999624,1.290200,1.573873,1.583603,1.184410,1.090997,1.060624,1.043204,1.194069,1.567379,1.400124,1.187711,1.124349,1.113409,1.090218,1.206241,1.428213,1.466147
3,0.408730,0.359662,0.331148,0.332599,0.407499,0.433837,0.602332,0.402484,0.428838,0.420318,0.374566,0.360211,0.422587,0.362438,0.368652,0.373735,0.383930,0.394509,0.395899,0.455351,0.481506,0.392743,0.369007,0.370575,0.372364,0.406188,0.461010,0.474532
4,0.422404,0.430875,0.449946,0.520700,0.453983,0.622715,0.608194,0.479387,0.496449,0.435287,0.424406,0.440619,0.651719,0.527341,0.486947,0.455321,0.453260,0.450625,0.463734,0.625805,0.650339,0.482945,0.457457,0.470726,0.482871,0.496654,0.604415,0.604536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12191,0.128544,0.133970,0.139405,0.140420,0.131640,0.194804,0.157752,0.140396,0.156087,0.155214,0.138554,0.158417,0.162208,0.109328,0.124815,0.128203,0.134872,0.140703,0.132760,0.147825,0.130593,0.117594,0.121420,0.127832,0.134449,0.133994,0.165792,0.152036
12192,0.206858,0.195634,0.188470,0.211799,0.307360,0.396261,0.305189,0.250490,0.225812,0.219734,0.255343,0.263959,0.351914,0.344115,0.231780,0.219014,0.218169,0.216522,0.284538,0.336489,0.341310,0.233627,0.217282,0.211336,0.218080,0.302132,0.351668,0.340154
12193,0.199442,0.200549,0.202226,0.199433,0.242613,0.334718,0.305292,0.212616,0.207093,0.199566,0.237868,0.266541,0.302887,0.346094,0.214026,0.207233,0.204759,0.211739,0.289037,0.343196,0.324581,0.207095,0.198362,0.199624,0.203499,0.278499,0.285667,0.282954
12194,0.116540,0.113957,0.118031,0.110473,0.129658,0.165444,0.144743,0.146232,0.143333,0.139504,0.167418,0.173569,0.175563,0.164022,0.145093,0.143160,0.142121,0.140556,0.161535,0.170858,0.160630,0.136316,0.132347,0.135679,0.139172,0.164795,0.173509,0.163122


In [68]:
CA_1_eva

,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
1612,2,0,0,0,0,1,1,0,6,1,0,3,0,0,0,2,0,0,2,3,1,0,0,0,1,0,0,0
4661,0,3,0,0,0,1,4,0,0,0,0,0,0,0,1,2,1,0,1,0,0,1,1,0,0,1,2,0
7710,1,0,1,0,8,1,0,0,1,0,0,1,1,0,0,0,0,0,1,2,2,0,0,1,0,3,2,2
10759,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0
1613,0,0,0,0,0,1,0,1,3,0,0,1,0,2,1,0,0,1,2,1,0,1,1,1,0,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10757,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1611,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
4660,0,2,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,3,0
7709,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0


In [69]:
from typing import Union

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm_notebook as tqdm


class WRMSSEEvaluator(object):

    def __init__(self, train_df: pd.DataFrame, valid_df: pd.DataFrame, calendar: pd.DataFrame, prices: pd.DataFrame):
        train_y = train_df.loc[:, train_df.columns.str.startswith('d_')]
        train_target_columns = train_y.columns.tolist()
        weight_columns = train_y.iloc[:, -28:].columns.tolist()

        train_df['all_id'] = 0  # for lv1 aggregation

        id_columns = train_df.loc[:, ~train_df.columns.str.startswith('d_')].columns.tolist()
        valid_target_columns = valid_df.loc[:, valid_df.columns.str.startswith('d_')].columns.tolist()

        if not all([c in valid_df.columns for c in id_columns]):
            valid_df = pd.concat([train_df[id_columns], valid_df], axis=1, sort=False)
            #valid_df = valid_df.loc[:,~valid_df.columns.duplicated()]

        self.train_df = train_df
        self.valid_df = valid_df
        self.calendar = calendar
        self.prices = prices

        self.weight_columns = weight_columns
        self.id_columns = id_columns
        self.valid_target_columns = valid_target_columns

        weight_df = self.get_weight_df()


        self.group_ids = (
            'all_id',
            'state_id',
            'store_id',
            'cat_id',
            'dept_id',
            ['state_id', 'cat_id'],
            ['state_id', 'dept_id'],
            ['store_id', 'cat_id'],
            ['store_id', 'dept_id'],
            'item_id',
            ['item_id', 'state_id'],
            ['item_id', 'store_id']
        )
        print("start")
        for i, group_id in enumerate(tqdm(self.group_ids)):
            train_y = train_df.groupby(group_id)[train_target_columns].sum()
            scale = []
            for _, row in train_y.iterrows():
                series = row.values[np.argmax(row.values != 0):]
                scale.append(((series[1:] - series[:-1]) ** 2).mean())
            
            setattr(self, f'lv{i + 1}_scale', np.array(scale))
            setattr(self, f'lv{i + 1}_train_df', train_y)
            print(group_id)
            setattr(self, f'lv{i + 1}_valid_df', valid_df.groupby(group_id)[valid_target_columns].sum())

            lv_weight = weight_df.groupby(group_id)[weight_columns].sum().sum(axis=1)
            setattr(self, f'lv{i + 1}_weight', lv_weight / lv_weight.sum())

    def get_weight_df(self) -> pd.DataFrame:
        day_to_week = self.calendar.set_index('d')['wm_yr_wk'].to_dict()
        weight_df = self.train_df[['item_id', 'store_id'] + self.weight_columns].set_index(['item_id', 'store_id'])
        weight_df = weight_df.stack().reset_index().rename(columns={'level_2': 'd', 0: 'value'})
        weight_df['wm_yr_wk'] = weight_df['d'].map(day_to_week)

        weight_df = weight_df.merge(self.prices, how='left', on=['item_id', 'store_id', 'wm_yr_wk'])
        weight_df['value'] = weight_df['value'] * weight_df['sell_price']
        weight_df = weight_df.set_index(['item_id', 'store_id', 'd']).unstack(level=2)['value']
        weight_df = weight_df.loc[zip(self.train_df.item_id, self.train_df.store_id), :].reset_index(drop=True)
        weight_df = pd.concat([self.train_df[self.id_columns], weight_df], axis=1, sort=False)
        return weight_df

    def rmsse(self, valid_preds: pd.DataFrame, lv: int) -> pd.Series:
        valid_y = getattr(self, f'lv{lv}_valid_df')
        score = ((valid_y - valid_preds) ** 2).mean(axis=1)
        scale = getattr(self, f'lv{lv}_scale')
        return (score / scale).map(np.sqrt)

    def score(self, valid_preds: Union[pd.DataFrame, np.ndarray]) -> float:
        assert self.valid_df[self.valid_target_columns].shape == valid_preds.shape

        if isinstance(valid_preds, np.ndarray):
            valid_preds = pd.DataFrame(valid_preds, columns=self.valid_target_columns)

        valid_preds = pd.concat([self.valid_df[self.id_columns], valid_preds], axis=1, sort=False)

        all_scores = []
        for i, group_id in enumerate(self.group_ids):
            lv_scores = self.rmsse(valid_preds.groupby(group_id)[self.valid_target_columns].sum(), i + 1)
            weight = getattr(self, f'lv{i + 1}_weight')
            lv_scores = pd.concat([weight, lv_scores], axis=1, sort=False).prod(axis=1)
            all_scores.append(lv_scores.sum())

        return np.mean(all_scores)

In [70]:
#train_df = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_validation.csv')
#train_fold_df = train_df.iloc[:, :-28]
#valid_fold_df = train_df.iloc[:, -28:]
#valid_preds = valid_fold_df.copy() + np.random.randint(100, size=valid_fold_df.shape)
calendar = pd.read_csv("calendar.csv")
prices = pd.read_csv("sell_prices.csv")

In [71]:
train_fold_df = evaluation_set.iloc[:, :-28]
train_fold_df = train_fold_df.sort_values('id')

In [72]:
mask = train_fold_df["state_id"] == "CA"
train_fold_df = train_fold_df[mask]

In [73]:
train_fold_df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,...,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1612,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,0,0,1,4,2,0,2,0,0,0,0,3,1,3,0,2,1,2,...,0,4,0,0,4,1,3,0,1,0,2,2,0,1,1,0,2,0,4,1,1,0,1,1,0
4661,FOODS_1_001_CA_2_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_2,CA,2,0,0,0,1,0,8,0,4,1,2,1,1,3,4,1,2,2,2,...,0,0,0,0,1,1,2,0,0,0,0,0,0,2,0,1,0,14,0,1,1,4,0,0,4
7710,FOODS_1_001_CA_3_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_3,CA,1,2,1,1,1,2,0,1,1,1,0,0,3,3,2,7,1,2,4,...,0,0,1,0,0,0,1,0,0,4,2,1,0,0,0,0,0,13,0,0,0,0,0,1,0
10759,FOODS_1_001_CA_4_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_4,CA,0,1,1,1,1,1,0,0,0,0,0,0,1,2,1,0,1,0,0,...,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,1,1,1
1613,FOODS_1_002_CA_1_evaluation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,0,1,0,1,1,1,0,0,0,0,1,1,0,2,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,3,1,0,0,1,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10757,HOUSEHOLD_2_515_CA_4_evaluation,HOUSEHOLD_2_515,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0
1611,HOUSEHOLD_2_516_CA_1_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,CA_1,CA,0,1,0,0,1,0,0,1,0,2,0,1,0,1,0,0,0,1,0,...,0,0,0,2,0,0,0,0,2,1,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0
4660,HOUSEHOLD_2_516_CA_2_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,CA_2,CA,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
7709,HOUSEHOLD_2_516_CA_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,CA_3,CA,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [74]:
train_fold_df.shape

(12196, 1919)

In [75]:
CA_1_eva.shape

(12196, 28)

In [76]:
pred_period1.shape

(12196, 28)

In [77]:
CA_1_eva.columns

Index(['d_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919', 'd_1920',
       'd_1921', 'd_1922', 'd_1923', 'd_1924', 'd_1925', 'd_1926', 'd_1927',
       'd_1928', 'd_1929', 'd_1930', 'd_1931', 'd_1932', 'd_1933', 'd_1934',
       'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939', 'd_1940', 'd_1941'],
      dtype='object')

In [78]:
train_fold_df.reset_index(drop = True)
CA_1_eva.reset_index(drop = True)
evaluator = WRMSSEEvaluator(train_fold_df.reset_index(drop = True), CA_1_eva.reset_index(drop = True), calendar, prices)
evaluator.score(pred_period1.reset_index(drop = True))

start


all_id
state_id
store_id
cat_id
dept_id
['state_id', 'cat_id']
['state_id', 'dept_id']
['store_id', 'cat_id']
['store_id', 'dept_id']
item_id
['item_id', 'state_id']
['item_id', 'store_id']



0.47831233604143203

In [98]:
0.47831233604143203

0.44529593426108205